In [8]:
import httplib2
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.credentials import Credentials
from datetime import datetime
import json

In [2]:

CREDENTIALS_FILE = 'creds.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    CREDENTIALS_FILE,
    ['https://www.googleapis.com/auth/spreadsheets',
     'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())

service = build('sheets', 'v4', http = httpAuth)
service_drive = build('drive', 'v3', http = httpAuth)

spreadsheet_id = '1ZJ6VzQOQJlZvVtJ2UNhb8Uj4El2TV8lG7wMfMwMiJaM'

def get_table_modified_time():
  response = service_drive.files().get(fileId=spreadsheet_id,fields="modifiedTime").execute()
  modified_time_str = response['modifiedTime']
  date = datetime.fromisoformat(modified_time_str[:-1])
  return date

def get_content():
  values = service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    
    range='A1:E100',
    majorDimension='ROWS'
  ).execute()
  
  keys = values['values'][0]
  keys = [key.lower() for key in keys]
  videos_row = values['values'][1:]
  videos = []
  
  for row in videos_row:
    video = {}
    for index, value in enumerate(keys):
      if len(row) > index:
          video[value] = row[index]
      else:
          video[value] = ''
        
    if 'link' in video:
      duration = get_video_length(video['link'])
      video['duration'] = duration
      
    videos.append(video)
  
  data = []
  topic_index_dict = {}
  for video in videos:
    topic = video['topic']
    if topic in topic_index_dict:
      index = topic_index_dict[topic]
      data[index]['videos'].append(video)
    else:
      topic_index_dict[topic] = len(data)
      data.append({
        'topic': topic,
        'videos': [
          video
        ]
      })
  return data

In [6]:
sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()

properties = sheet_metadata.get('sheets')
for item in properties:
    print(item)
    if item.get("properties").get('title') == 'SHEET_TITILE':
        sheet_id = (item.get("properties").get('sheetId'))

print (sheet_id)

{'properties': {'sheetId': 0, 'title': 'УникальныеСообщения', 'index': 0, 'sheetType': 'GRID', 'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}
{'properties': {'sheetId': 1443323139, 'title': 'УпражненияЭмоции', 'index': 1, 'sheetType': 'GRID', 'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}
0


In [11]:
values = service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    range='УникальныеСообщения!A1:B100',
    majorDimension='ROWS'
).execute()

values = values['values']

In [12]:
uniqueMessages = {}
for line in values:
    uniqueMessages[line[0]] = line[1]
uniqueMessages

{'startMessage': 'Привет👋, я Забот. Твой помощник в пространстве неопределенности и тревоги.\n\nЯ помогу тебе преодолеть эмоциональный шторм, успокоиться, снять стресс и позаботиться о себе и близких, чтобы ты мог жить полноценной жизнью и сохранять продуктивность даже в эти непростые времена.\n\nЭто анонимно.\n\n✅Для начала напиши start'}

In [ ]:
from pathlib import Path

In [ ]:

class PathConfig:
    storageBaseDir = Path("./DataStorage")
    storageUsersDir = storageBaseDir / "Users"

path = PathConfig()

In [ ]:

if path.storageUsersDir.is_dir():
    print("Is dir")
else:
    print("Is not dir")

In [ ]:
kek = path.storageUsersDir / "azazazadsdz"

In [ ]:
kek.mkdir(parents=True, exist_ok=True)

In [ ]:
import pytz
from datetime import datetime
# import datetime

In [ ]:
tz = pytz.timezone('Europe/Berlin')
now = datetime.now(tz)

In [ ]:
def getTimestamp():
    tz = pytz.timezone('Europe/Moscow')
    now = datetime.now(tz)
    return now.strftime("%d.%m.%Y, %H:%M:%S")

In [ ]:
getTimestamp()

In [ ]:
datetime.now().isocalendar()[1]

In [ ]:
datetime(2022, 1, 3).isocalendar()[1]

In [ ]:
import xlsxwriter
import json
from pathlib import Path
from datetime import date, timedelta


In [ ]:
class PathConfig:
    storageBaseDir = Path("./DataStorage")
    storageUsersDir = storageBaseDir / "Users"
    storageBotContentDir = storageBaseDir / "BotContent"
    storageBotContentPrivateConfig = storageBotContentDir / "PrivateConfig.json"

path = PathConfig()

In [ ]:
def getJsonData(filePath):
    with filePath.open() as json_file:
        data = json.load(json_file)
    return data

In [ ]:

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days + 1)):
        yield start_date + timedelta(n)

In [ ]:
dateConfig = getJsonData(path.storageBotContentPrivateConfig)["startDate"]
startDate = date(dateConfig["year"], dateConfig["month"], dateConfig["day"])

In [ ]:
# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('TotalHistory.xlsx')
worksheet = workbook.add_worksheet("Start")
eventName = "Старт"

row = 0 
col = 0

worksheet.write(row, col, "Дата / Пользователь")
row += 1

dates = [strDate.strftime("%d.%m.%Y") for strDate in daterange(startDate, date.today())]
for single_date in dates:
    worksheet.write(row, col, single_date)
    row += 1

col += 1

for userFolder in path.storageUsersDir.iterdir():
    row = 0
    history = getJsonData(userFolder / "history.json")
    worksheet.write(row, col, userFolder.name)
    
    row = 1
    for single_date in dates:
        dateEventsCount = len([event for event in history if event["timestamp"]["date"] == single_date and event["event"] == eventName])
        worksheet.write(row, col, dateEventsCount)
        row += 1

    col += 1

workbook.close()

In [ ]:
path.storageUsersDir.absolute()

In [ ]:
glob("DataStorage/Users//*")

In [3]:
import enum

In [4]:
class HistoryEvent(enum.Enum):

    start = "Старт"
    sendMessage = "Отправил сообщение"

In [10]:
kek: HistoryEvent = None

In [14]:
kek is not None

False

In [12]:
type(kek)

NoneType

In [6]:
for kek in HistoryEvent:
    print(kek)

HistoryEvent.start
HistoryEvent.sendMessage


In [10]:
json.dumps({ "kek": 34234 })

'{"kek": 34234}'

In [11]:
kek = ["kek", "kek"]

In [13]:
len(kek)

2

In [15]:
kek = {
    "key1": "value1",
    "key2": "value2",
    "key3": "value3"
}

In [19]:
for givno in kek:
    print(givno)

key1
key2
key3
